# VCノード用仮想マシンセットアップ

この Notebook では、mdx REST API にアクセスするためのPythonクライアントライブラリを使用し、mdx仮想マシンの作成と、[ocs-template](https://github.com/nii-gakunin-cloud/ocs-templates)にてVCノードとして利用するためのセットアップを行います。

## 準備

1. mdx REST client for python インストール
1. mdx REST API 認証トークン設定
1. VCコントローラ 認証トークン設定
1. mdx VM にSSHログインするためのキーペア作成
1. mdx の REST API エンドポイントに接続できることの確認

### mdx REST client for python インストール

In [ ]:
!pip install --user git+https://github.com/nii-gakunin-cloud/mdx-rest-client-python.git

### mdx REST API 認証トークン設定

mdx REST API 認証トークンは、[mdxユーザポータル](https://oprpl.mdx.jp/) の「トークン発行」により取得します。  
「トークン発行」は、ユーザポータル画面右上にあるユーザ名をクリックすると表示されます。

In [ ]:
# mdx REST API 認証トークン設定
from getpass import getpass
mdx_token = getpass()

### VCコントローラ 認証トークン設定

vcpsdkを利用してVCノードの設定を行うため、vcpsdkの公開鍵をノードに登録します。  
以下のセルを実行し、トークンを入力します。  
その後、トークンが正しいこと・VCコントローラが利用できることを確認します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

In [ ]:
from vcpsdk.vcpsdk import VcpSDK
sdk = VcpSDK(vcc_access_token)
sdk_pubkey = sdk.get_publickey()

### パスワード設定

OSのログインパスワードを設定します。  
以下のセルを実行すると、ランダムなパスワードを設定します。  
任意のパスワードを設定しても構いません。

In [ ]:
import string
import secrets
alphabet = string.ascii_letters + string.digits
randompassword = ''.join(secrets.choice(alphabet) for i in range(8))

# 設定するOSパスワード
# 任意の物に変更してください。変更しないランダムなパスワードが設定されるため、変数の内容を自身で別途保存してください。
mdx_user_password = randompassword

### mdx VM への SSH ログイン用キーペア作成

mdx VM デプロイ時の設定項目に含まれる公開鍵を用意します。

In [ ]:
!mkdir -p -m 700 ~/.ssh
!test -f ~/.ssh/id_ed25519 || ssh-keygen -t ed25519 -f ~/.ssh/id_ed25519 -N ""

### mdx の REST API エンドポイントに接続できることの確認

デフォルトのresolverがIPv6のアドレスを返すことにより以降のAPIで接続不可の場合があるため、以下のコードを実行しておきます。  
その後、HTTPステータスコード 200 が返ることを確認します。

In [ ]:
from mdx.mdx_ext import use_ipv4_only
use_ipv4_only()

In [ ]:
!curl https://oprpl.mdx.jp -w '%{http_code}\n' -o /dev/null -s

## mdx 仮想マシンの作成

1. mdx REST Client for Python ライブラリの読み込み
2. mdx VM作成に必要なパラメータ設定
3. mdx VMデプロイ実行

### mdx REST Client for Python ライブラリの読み込み

In [ ]:
from mdx.mdx_ext import MdxResourceExt
mdx = MdxResourceExt(mdx_token)

### mdx VM作成に必要なパラメータ設定

- プロジェクトID
- ネットワークセグメントID
- sshログインのための公開鍵

自身が所属している（利用可能な）mdxのプロジェクト情報を確認します。

In [ ]:
import json
projects = mdx.get_assigned_projects()
print(json.dumps(projects[0]["projects"], indent=2))

以降のmdx操作対象とする「プロジェクト名」 (`name`) を設定します。

In [ ]:
project_name = 
mdx.set_current_project_by_name(project_name)

操作対象として設定したプロジェクト情報を確認します。

In [ ]:
print(json.dumps(mdx.get_current_project(), indent=2))

プロジェクトで利用可能なネットワークセグメントのリストを取得し、先頭のIDを設定します。

In [ ]:
segments = mdx.get_segments()
print(json.dumps(segments, indent=2))

segment_id = mdx.get_segments()[0]["uuid"]
print(segment_id)

sshログインのための公開鍵ファイルの内容を設定します。

In [ ]:
import os
with open(os.path.expanduser('~/.ssh/id_ed25519.pub')) as f:
    ssh_shared_key = f.read()
print(ssh_shared_key)

### mdx VMデプロイ

仮想マシンを起動します。デプロイ完了後、VM情報を出力します。  

In [ ]:
# Ubuntu22.04イメージを使用します。
DEFAULT_TEMPLATE_NAME = "UT-20250205-1029-ubuntu-2204-server"
DEFAULT_CATALOG = "32da6d82-bdca-405e-9209-62044bd92923"
# [1-1] のように範囲指定を行うと、指定した数仮想マシンが起動します。
vm_name = "vm-for-vcnode-[1-1]"

mdx_spec = dict(
    catalog=DEFAULT_CATALOG,
    template_name=DEFAULT_TEMPLATE_NAME,
    pack_num=3,
    pack_type="cpu",
    disk_size=40,
    gpu="0",
    network_adapters=[
        dict(
            adapter_number=1,
            segment=segment_id
        )
    ],
    shared_key=ssh_shared_key,
    storage_network="portgroup",
    service_level="spot",
)

vm_infos = mdx.deploy_vm(vm_name, mdx_spec)
print(json.dumps(vm_infos, indent=2, ensure_ascii=False))

### VCノード用セットアップ

各種セットアップ用スクリプトを定義しています。  
以下のセルを実行することで、利用可能となります。

In [ ]:
import pexpect
import tempfile
import subprocess

mdx_change_addr_script = """
#!/bin/bash

set -euo pipefail

netif=ens160
newaddr=$1

dns1=`resolvectl dns ${netif} | awk '{print $4}'`
dns2=`resolvectl dns ${netif} | awk '{print $5}'`
defroute=`ip route list default | awk '{print $3}'`

sudo chmod -R 600 /etc/netplan/

echo "DNS=${dns1}" | sudo tee -a /etc/systemd/resolved.conf
if [ -n "${dns2}" ]; then
    echo "FallbackDNS=${dns2}" | sudo tee -a /etc/systemd/resolved.conf
fi
sudo systemctl restart systemd-resolved

masklen=`ip addr show dev $netif | \
    awk '/inet / {i = index($2, "/"); print(substr($2, i+1));}'`
sudo netplan set ethernets.${netif}.addresses=[${newaddr}/${masklen}]
sudo netplan set ethernets.${netif}.routes="[{to: default, via: ${defroute} }]"
sudo netplan set ethernets.ens192.dhcp-identifier=mac
sudo netplan set ethernets.ens192.dhcp4=true
sudo rm -f /etc/netplan/mdx.yaml
sudo netplan apply
"""

vcnode_setup_script = """
#!/bin/sh
sudo apt-get -qq update
sudo apt-get -qq install -y ca-certificates curl gnupg lsb-release
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu \
  $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get -qq update
sudo apt-get -qq install -y docker-ce
echo 'Port 20022' | sudo tee -a /etc/ssh/sshd_config
sudo systemctl restart sshd
"""

cgroupv2tov1 = """
#!/bin/bash

set -o errexit
cgroups=${VCP_CGROUPS:-"v1"}
if [[ "$cgroups" == "v2" ]]; then
    exit 0
fi
if [[ "$(stat -fc %T /sys/fs/cgroup/)" == "tmpfs" ]]; then
    exit 0
fi
GRUB_FILE="/etc/default/grub"
PARAM="systemd.unified_cgroup_hierarchy"
PARAM_CGROUP="0"
current_GRUB_CMDLINE_LINUX=$(grep "GRUB_CMDLINE_LINUX" $GRUB_FILE | sed -n 's/GRUB_CMDLINE_LINUX=//p' | sed 's/^.*"\(.*\)".*$/\1/')

if grep -q "$PARAM" "$GRUB_FILE"; then
    if grep -q "$PARAM=$PARAM_CGROUP" "$GRUB_FILE"; then
        echo "Already cgroupv1"
    else
        echo "Option defined but not cgroupv1"
        sed -i "s/GRUB_CMDLINE_LINUX.*/GRUB_CMDLINE_LINUX=\"$PARAM=$PARAM_CGROUP\"/" $GRUB_FILE
    fi
else
    if [ ${#current_GRUB_CMDLINE_LINUX} -eq 0 ]; then
        cgroupv1_GRUB_CMDLINE_LINUX="$current_GRUB_CMDLINE_LINUX $PARAM=$PARAM_CGROUP"
    else
        cgroupv1_GRUB_CMDLINE_LINUX="$PARAM=$PARAM_CGROUP"
    fi
    cgroupv1_GRUB_CMDLINE_LINUX=$(echo "$cgroupv1_GRUB_CMDLINE_LINUX" | sed 's/^ *//')
    cgroupv1_GRUB_CMDLINE_LINUX="\"$cgroupv1_GRUB_CMDLINE_LINUX\""
    sed -i "s/GRUB_CMDLINE_LINUX.*/GRUB_CMDLINE_LINUX=$cgroupv1_GRUB_CMDLINE_LINUX/" $GRUB_FILE
fi

echo $(cat $GRUB_FILE)

update-grub
shutdown --reboot now
"""

def exec_ssh(host, user, cmd, ssh_key, port=22, ssh_opt: list = None, timeout=30):
    _ssh_opt = ["StrictHostKeyChecking=no", "UserKnownHostsFile=/dev/null"]
    if ssh_opt:
        _ssh_opt.extend(ssh_opt)
    _ssh_opt_cmd = list()
    for o in _ssh_opt:
        _ssh_opt_cmd.append('-o')
        _ssh_opt_cmd.append(o)
    result = subprocess.run(['ssh', *_ssh_opt_cmd, '-p', str(port), '-i', os.path.expanduser(ssh_key),
                             f'{user}@{host}', cmd], capture_output=True, timeout=timeout)
    return result

def exec_scp(host, user, local_path, remote_path, ssh_key, port=22, ssh_opt: list = None, timeout=30):
    _ssh_opt = ["StrictHostKeyChecking=no", "UserKnownHostsFile=/dev/null"]
    if ssh_opt:
        _ssh_opt.extend(ssh_opt)
    _ssh_opt_cmd = list()
    for o in _ssh_opt:
        _ssh_opt_cmd.append('-o')
        _ssh_opt_cmd.append(o)
    result = subprocess.run(['scp', *_ssh_opt_cmd, '-P', str(port), '-i', os.path.expanduser(ssh_key),
                             local_path, f'{user}@{host}:{remote_path}'], capture_output=True, timeout=timeout)
    return result

def get_hostname(host, user, ssh_key='~/.ssh/id_ed25519', port=22, ssh_opt=None, timeout=30):
    cmd = "sudo hostname"
    return [exec_ssh(host, user, cmd, ssh_key, port, ssh_opt, timeout).stdout.decode('utf-8')]

def set_hostname(host, user, hostname, ssh_key='~/.ssh/id_ed25519', port=22, ssh_opt=None, timeout=30):
    cmd = f"sudo hostnamectl set-hostname {hostname}"
    res = exec_ssh(host, user, cmd, ssh_key, port, ssh_opt, timeout)
    return [res.stdout.decode('utf-8')]

def register_pubkey(host, user, pubkey, ssh_key='~/.ssh/id_ed25519', port=22, ssh_opt=None, timeout=30):
    cmd = f"echo {pubkey} >> ~/.ssh/authorized_keys"
    return [exec_ssh(host, user, cmd, ssh_key, port, ssh_opt, timeout).stdout.decode('utf-8')]

def fix_ipaddr(host, user, ip_addr, ssh_key='~/.ssh/id_ed25519', port=22, ssh_opt=None, timeout=30):
    remote_script = "/home/mdxuser/mdx_change_addr.sh"
    with tempfile.NamedTemporaryFile("w", delete=False) as tmp:
        tmp.write(mdx_change_addr_script)
    try:
        res_scp = exec_scp(host, user, tmp.name, remote_script, ssh_key, port, ssh_opt)
        res_ssh = exec_ssh(host, user, f"chmod +x {remote_script} && sudo bash {remote_script} {ip_addr}", ssh_key, port, ssh_opt, timeout)
    finally:
        os.remove(tmp.name)
    return [res_scp.stdout.decode('utf-8'), res_ssh.stdout.decode('utf-8')]

def setup_for_vcnode(host, user, ssh_key='~/.ssh/id_ed25519', port=22, ssh_opt=None, timeout=60):
    remote_script = "/home/mdxuser/vcnode_setup_script.sh"
    with tempfile.NamedTemporaryFile("w", delete=False) as tmp:
        tmp.write(vcnode_setup_script)
    try:
        res_scp = exec_scp(host, user, tmp.name, remote_script, ssh_key, port, ssh_opt)
        res_ssh = exec_ssh(host, user, f"chmod +x {remote_script} && sudo bash {remote_script}", ssh_key, port, ssh_opt, timeout)
    finally:
        os.remove(tmp.name)
    return [res_scp.stdout.decode('utf-8'), res_ssh.stdout.decode('utf-8')]

def cgroup_v2tov1(host, user, ssh_key='~/.ssh/id_ed25519', port=22, ssh_opt=None, timeout=60):
    remote_script = "/home/mdxuser/cgroupv2tov1.sh"
    with tempfile.NamedTemporaryFile("w", delete=False) as tmp:
        tmp.write(cgroupv2tov1)
    try:
        res_scp = exec_scp(host, user, tmp.name, remote_script, ssh_key, port, ssh_opt)
        res_ssh = exec_ssh(host, user, f"chmod +x {remote_script} && sudo bash {remote_script}", ssh_key, port, ssh_opt, timeout)
    finally:
        os.remove(tmp.name)
    return [res_scp.stdout.decode('utf-8'), res_ssh.stdout.decode('utf-8')]


定義したセットアップスクリプトを利用して、デプロイした仮想マシンをセットアップします。

In [ ]:
for vm_info in vm_infos:
    ip_addr = vm_info["service_networks"][0]["ipv4_address"][0]
    ssh_key='~/.ssh/id_ed25519'
    username="mdxuser"
    try:
        mdx.set_first_password(ip_addr, mdx_user_password, ssh_key=ssh_key, username=username)
    except Exception:
        # 既に設定済みとして扱う
        pass
    print(set_hostname(ip_addr, username, vm_info['name'], ssh_key=ssh_key))
    print(register_pubkey(ip_addr, username, sdk_pubkey, ssh_key=ssh_key))
    print(fix_ipaddr(ip_addr, username, ip_addr, ssh_key=ssh_key))
    print(setup_for_vcnode(ip_addr, username, ssh_key=ssh_key))
    print(cgroup_v2tov1(ip_addr, username, ssh_key=ssh_key, port=20022))


In [ ]:
import time
# cgroupの変更のためのシャットダウンが実行されるまでラグがあるため待機する
time.sleep(60)
# sshアクセス可能になるまで待機する
hostname = get_hostname(ip_addr, username, ssh_key=ssh_key, port=20022, timeout=600)[0]
if not hostname:
    hostname = get_hostname(ip_addr, username, ssh_key=ssh_key, port=20022, timeout=600)[0]
print(hostname)